In [1]:
import numpy as np
import pandas as pd

In [57]:
# 변수명, 측정값, 최솟값, 최댓값, 망대 여부, 퍼지 여부, 배점
## 측정값, 최솟값, 최댓값은 임의로 작성 ##
eval_data = [
             ['납기 준수율', 90, 0, 100, True, False, 13],
             ['평균 납기지체일수', 4, 0, 7, False, False, 12],
             ['하자처리 건수', 3, 0, 8, False, False, 9],
             ['품질 만족도', (80, 87), (0, 5), (95, 100), True, True, 12],
             ['가격 만족도', (68, 75), (0, 5), (95, 100), True, True, 6],
             ['서비스 만족도', (85, 90), (0, 5), (95, 100), True, True, 6],
             ['사후 만족도', (75, 90), (0, 5), (95, 100), True, True, 6],
             ['납품품목비율', 95, 0, 100, True, False, 8],
             ['하자처리 기간', 4, 3, 24, False, False, 8],
             ['부정당업자 제재기간', 16, 0, 270, False, False, 10],
             ['거래정지 지간', 10, 0, 270, False, False, 10]
            ]

In [58]:
# 입력 데이터
pd.DataFrame(eval_data, columns=['항목', '측정값', '최솟값', '최댓값', '망대 여부', 'Fuzzy 여부', '배점'])

,항목,측정값,최솟값,최댓값,망대 여부,Fuzzy 여부,배점
0,납기 준수율,90,0,100,True,False,13
1,평균 납기지체일수,4,0,7,False,False,12
2,하자처리 건수,3,0,8,False,False,9
3,품질 만족도,"(80, 87)","(0, 5)","(95, 100)",True,True,12
4,가격 만족도,"(68, 75)","(0, 5)","(95, 100)",True,True,6
5,서비스 만족도,"(85, 90)","(0, 5)","(95, 100)",True,True,6
6,사후 만족도,"(75, 90)","(0, 5)","(95, 100)",True,True,6
7,납품품목비율,95,0,100,True,False,8
8,하자처리 기간,4,3,24,False,False,8
9,부정당업자 제재기간,16,0,270,False,False,10


In [59]:
# 구간 연산 - 뺄셈 함수: intv1 - intv2
def intv_sub(intv1, intv2):
    min_ = intv1[0] - intv2[1]
    max_ = intv1[1] - intv2[0]
    return min_, max_

In [60]:
# 각 항목의 fuzzy 만족도 지수를 계산하는 함수
def get_fuzzy_score(data):
    y_i = data[1]                             # 측정값
    y_min = data[2]; y_max = data[3]          # 최솟값; 최댓값
    isMangdae = data[4]; isFuzzy = data[5]    # 망대 여부; Fuzzy 여부
    
    if isMangdae:    # 망대 특성인 경우
        if isFuzzy:  # Fuzzy 데이터인 경우
            # elementwise 연산를 위해 nunmpy array로 변환
            y_min = np.array(y_min)
            y_max = np.array(y_max)
            y_i = np.array(y_i)
             
            if np.mean(y_i) < np.mean(y_min):
                score = (0, 0)
            elif np.mean(y_i) > np.mean(y_max):
                score = (1, 1)
            else:
                bunja = intv_sub(y_i, y_min)
                bunmo = intv_sub(y_max, y_min)
                cases = [a/b for a in bunja for b in bunmo]       # 분자/분모의 구간 연산
                score = (min(cases), max(cases))
                
            return score
        
        else:       # Crisp 데이터인 경우
            if y_i <= y_min:
                score = 0
            elif y_i >= y_max:
                score = 1
            else:
                score = (y_i - y_min) / (y_max - y_min)
            
            return (score, score)
     
    
    else:    # 망소 특성인 경우 -> 1에서 뺀 값을 반환
        if isFuzzy:  # Fuzzy 데이터인 경우
            # elementwise 연상를 위해 nunmpy array로 변환
            y_min = np.array(y_min)
            y_max = np.array(y_max)
            y_i = np.array(y_i)
            
            if np.mean(y_i) < np.mean(y_min):
                score = (1, 1)
            elif np.mean(y_i) > np.mean(y_max):
                score = (0, 0)
            else:
                bunja = intv_sub(y_i, y_min)
                bunmo = intv_sub(y_max, y_min)
                cases = [a/b for a in bunja for b in bunmo]        # 분자/분모의 구간 연산
                score = (1-max(cases), 1-min(cases))
            
            return score
        
        else:       # Crisp 데이터인 경우
            if y_i <= y_min:
                score = 0
            elif y_i >= y_max:
                score = 1
            else:
                score = (y_i - y_min) / (y_max - y_min)
            
            return (1-score, 1-score)

In [64]:
# 하나씩 확인
for data in eval_data:
    score = get_fuzzy_score(data)
    print(f'항목 : {data[0]},\t\t 만족도 지수 : ({score[0]:.3f}, {score[1]:.3f})')

항목 : 납기 준수율,		 만족도 지수 : (0.900, 0.900)
항목 : 평균 납기지체일수,		 만족도 지수 : (0.429, 0.429)
항목 : 하자처리 건수,		 만족도 지수 : (0.625, 0.625)
항목 : 품질 만족도,		 만족도 지수 : (0.750, 0.967)
항목 : 가격 만족도,		 만족도 지수 : (0.630, 0.833)
항목 : 서비스 만족도,		 만족도 지수 : (0.800, 1.000)
항목 : 사후 만족도,		 만족도 지수 : (0.700, 1.000)
항목 : 납품품목비율,		 만족도 지수 : (0.950, 0.950)
항목 : 하자처리 기간,		 만족도 지수 : (0.952, 0.952)
항목 : 부정당업자 제재기간,		 만족도 지수 : (0.941, 0.941)
항목 : 거래정지 지간,		 만족도 지수 : (0.963, 0.963)


In [62]:
# 종합 만족도 지수를 계산하는 함수
def get_total(eval_data):
    total_score = 0
    for data in eval_data:
        total_score += np.array(get_fuzzy_score(data)) * data[-1]
    
    total_score /= 100
    return total_score

In [63]:
print(get_total(eval_data))

[0.78503942 0.85323942]
